# GaN findTDE Analysis Example

In [1]:
import os
import glob
from pathlib import Path

import subprocess
import re
import fortranformat as ff
import pprint

import numpy as np
import pandas as pd

import matplotlib as mpl
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.io as pio
# import seaborn as sns

from findtde.tde_analysis import *

## VASP

In [2]:
vasp_path = Path.cwd() / 'GaN_VASP'
print(vasp_path)

/storage/work/ash5615/bin/findTDE/examples/GaN_VASP


In [8]:
vasp_ref_file = vasp_path / 'inp' / 'POSCAR'
pos_f = open(vasp_ref_file, 'r')
pos_lines = pos_f.readlines()
pos_f.close()

# open POSCAR file and read lattice vector lines into a list
ai = [pos_lines[2][1:-1], pos_lines[3][1:-1], pos_lines[4][1:-1]]

# convert lattice vectors line from POSCAR file from Fortran to a list
lat_vec_line = ff.FortranRecordReader('(3F22.16)')
vasp_lattice_vecs = np.array([lat_vec_line.read(ai[j]) for j in range(len(ai))])
vasp_params = np.array([np.linalg.norm(vasp_lattice_vecs[0, :]), np.linalg.norm(vasp_lattice_vecs[1, :]), np.linalg.norm(vasp_lattice_vecs[2, :])])

Each calculation direction is checked for common errors. A tuple of nested dictionaries is created with the first dictionary summarizing any errors found and the second dictionary showing all the checks performed. Each dictionary uses the direction pseudo/atom type and number (e.g., 1L_ga34) for the keys with secondary dictionaries as the values. These dictionaries show if the TDE is found, if the velocity vector correctly corresponds to the chosen atom type and index, if the velocity vector correctly corresponds to the chosen KE value (within a specified tolerance), and if the final temperature at the end of the CGM relaxation is sufficiently low (below a specified tolerance ratio).

In [9]:
check_find_tde_runs(tde_calc_dir=vasp_path, program='vasp', ke_tol=1, temp_tol=0.6)

({},
 {'1L_ga34': {'tde': [True],
   'knockout': [True, True, True, True, True, True],
   'ke': [True, True, True, True, True, True],
   'temp': [True, True, True, True, True, True]}})

The data from all direction calculations in the directory is gathered into a single CSV file. The final energy difference from the energy of the perfect supercell calculation is also computed, where the perfect supercell energy is read from the OUTCAR file in the perfect directory.

In [10]:
tde_data_gather(ofile=(vasp_path / 'all_tde_data.csv'), tde_calc_dir=vasp_path)

The analysis function returns a tuple with two dictionaries. The first dictionary uses the knockout atom type and number (e.g., ga34) as the keys and tuples containing two pandas DataFrames as the values. The first DataFrame is the final energy for each calculation direction at each simulated kinetic energy, and the second DataFrame is the same but for the final energy difference from the perfect crystal energy (calculated in the data gather step). The second dictionary contains the keys corresponding the lattice direction pseudos (e.g., 1L) to the associated knockout directions (e.g., \[-1 4 1\])

In [10]:
all_find_tde_dfs, pseudo_keys = find_tde_analysis(['ga'], [34], datafile=(vasp_path / 'all_tde_data.csv'), keyfile=(vasp_path / 'latt_dirs_to_calc.csv'))

In [15]:
all_find_tde_dfs

{'ga34': (             1L
  25 -1823.366892
  33 -1823.366856
  34 -1811.867811
  35 -1811.868307
  37 -1811.829686
  41 -1811.869515,
             1L
  25   0.000913
  33   0.000949
  34  11.499994
  35  11.499498
  37  11.538119
  41  11.498290)}

In [16]:
pseudo_keys

{np.str_('1L'): array(['-1', '4', '1'], dtype='<U2')}

In [17]:
find_tde_df = all_find_tde_dfs['ga34'][1]

In [18]:
find_tde_df

,1L
25,0.000913
33,0.000949
34,11.499994
35,11.499498
37,11.538119
41,11.498290


The DataFrames can also be converted into arrays that associate the directions with the TDE values which are better suited for plotting purposes. Within this conversion, any runs above the kinetic energy cutoff can be removed, and the directions can be re-oriented if the if the reference structure does not correspond with the desired $x$-axis. In this example, the $a$ lattice vector is re-oriented along the $x$-axis.

In [19]:
tde_sph_arr, tde_pseudos = generate_tde_sph_arr(find_tde_df, pseudo_keys, lattice_vecs=vasp_lattice_vecs, e_tol=1.0, ke_cut=45, polar_offset=angle_between([1., 0., 0.], vasp_lattice_vecs[0]))

In [20]:
tde_sph_arr

array([[130.8900206,  77.96     ,  34.       ]])

In [21]:
tde_pseudos

dict_keys([np.str_('1L')])

In [ ]:
generate_tde_line_plot(find_tde_df, im_write=False, im_name='tde_lineplot.png')

In [ ]:
generate_tde_scatter_plot(tde_sph_arr, tde_pseudos, txt_show=False, im_write=False, im_name='tde_scatter.png')

In [24]:
ps_tde, ts_tde, es_tde = idw_heatmap(tde_sph_arr, RES=tde_sph_arr.shape[0], P=5)

/storage/work/ash5615/bin/.conda/envs/findtde/lib/python3.12/site-packages/findtde/utils.py:199: RuntimeWarning:

invalid value encountered in scalar divide



In [ ]:
generate_tde_heatmap_plot(ps_tde, ts_tde, es_tde, im_write=False, im_name='tde_heatmap.png')

Alternatively, the majority of analysis functions can be executed simultaneously using the evaluate_tdes function.

In [ ]:
evaluate_tdes('ga', 34, run_type='vasp', base_path=vasp_path, annotate_pseudos=False, interpolate_heatmap=False)

## LAMMPS

In [2]:
lmp_path = Path.cwd() / 'GaN_LAMMPS'
print(lmp_path)

/storage/work/ash5615/bin/findTDE/examples/GaN_LAMMPS


All analysis functions are compatible with LAMMPS as well as VASP, but some require specification of the program used within the function call. Again, the majority of analysis functions can be executed simultaneously using the evaluate_tdes function.

In [4]:
evaluate_tdes('ga', 34, run_type='lammps', base_path=lmp_path, annotate_pseudos=False, interpolate_heatmap=False)

{}


{'TDE': 100}


ValueError: could not convert string to float: np.str_('9S')